In [31]:
import pickle
import torch
from src.model import Transformer, TransformerConfig

In [33]:
with open("models/transformer_settings.pkl", "rb") as f:
    settings = pickle.load(f)
print(list(settings.keys()))

['vocabulary_size', 'max_input_length', 'batch_size', 'embedding_size', 'word_to_index', 'index_to_word', 'num_blocks', 'num_heads']


In [64]:
config = TransformerConfig(vocab_size=settings["vocabulary_size"], max_input_length=settings["max_input_length"], num_heads=settings["num_heads"], num_blocks=settings["num_blocks"], embedding_size=settings["embedding_size"])
transformer = Transformer(config)
transformer.load_state_dict(torch.load("models/transformer_model.pt", map_location=torch.device('cpu')))
transformer.train()
print("Model loaded")

Model loaded


In [65]:
index_to_word = settings["index_to_word"]

def generate_next_token(tokens=None):
    if tokens == None:
        tokens = []
    last_token = len(index_to_word)-1
    x = tokens + [last_token]*(settings["max_input_length"] - len(tokens))
    x = torch.LongTensor([x])
    with torch.no_grad():
        y = transformer(x)
    # Don't allow the model to generate <unknown> tokens
    y = y[:, :, :y.shape[2]-1]
    pred = y.argmax(dim=-1).view(-1)
    next_word = pred[len(tokens)].item()
    return next_word

def print_sentence(words):
    print(" ".join([index_to_word[word] for word in words]))

generate_next_token()

9

In [71]:
def generate_sentence(start=None):
    if start == None:
        sentence = []
    else:
        words = start.split(" ")
        sentence = [settings["word_to_index"][x] for x in words]
    
    while len(sentence) < config.max_input_length:
        next_word = generate_next_token(sentence)
        sentence += [next_word]
    
    print_sentence(sentence)

generate_sentence("and bring him if")
generate_sentence("a most acute juvenal volable and")
generate_sentence("the captive is enrich on whose side the beggar . the catastrophe")
generate_sentence("i say the polusion holds in")
    

and bring him if  him his be .  .            ,   ,  ,      ,  -
a most <unknown> juvenal <unknown> and of ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  , 
the captive is enrich on whose side the beggar . the <unknown> .  ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  ,  , 
i say the <unknown> holds in ,  ,           ,     ,   ,  ,  ,     


Ideas
- Better preprocessing: remove punctuation? remove apostrophes and comma remove unknown chars
- only parse dialogue?
- Different parameters?